## 0. 필요 라이브러리 import

In [ ]:
# !pip install google-cloud-storage

In [1]:
from google.oauth2 import service_account
from google.cloud import storage

## 1. GCP 권한 인증

In [ ]:
## 권한 인증

JSON_KEY_PATH = ".json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

In [ ]:
## Client 생성

## 2. Parquet 파일 다루기
- 데이터를 객체 저장소에 저장할 때는, 용량 효율성, 빅데이터 도구들과의 호환성, 파티셔닝 등의 이점 때문에 CSV나 JSON 같은 포맷보다 Parquet 포맷을 사용하는 경우가 많습니다. (대부분 Parquet 사용)

In [ ]:
## parquet 파일 읽기

import pandas as pd

df = pd.read_parquet("dataset/member.parquet")

df.head()

In [ ]:
## parquet 파일 쓰기 (데이터 프레임 -> Parquet 파일로 저장 압축하는거라 파라미터 몇개 더 던져 줘야함)

df.to_parquet(
    path='dataset/member_parquet.parquet',
    index=False,
    engine='pyarrow',
    compression='gzip'
)


## 3. Bucket 관련 작업

In [ ]:
## 1) 버킷 목록 가져오기

In [ ]:
## 2) 버킷 생성

In [ ]:
## 3) 버킷 정보 조회

In [ ]:
## 4) 버킷 삭제

## 4. 객체 관련 작업

In [ ]:
## 1) 객체 업로드 - 파일

In [ ]:
## 2) 객체 업로드 - DataFrame

In [ ]:
## 3) 객체 다운로드

In [ ]:
## 4) 폴더 내 객체 목록 가져오기

In [ ]:
## 5) 객체 유무 조회

In [ ]:
## 6) 객체 삭제

## 5. Pandas로 GCS 데이터 읽기
- GCS를 작업자의 로컬 디렉토리 처럼 활용하여 데이터를 조회할 수 있는 매우 편리한 기능!
1. 로컬 작업 디렉터리에 위치한 파일(read_csv, read_parquet)
2. 데이터베이스에 저장된 테이블 (read_sql)
3. 빅쿼리에 저장된 테이블(read_gbq)
4. 클라우드 객체 저장소(GCS)에 저장된 파일(read_csv, read_parquet)

In [ ]:
# !pip install fsspec gcsfs

In [ ]:
df = pd.read_parquet("dataset/member.parquet")

df.head()

,idx,sex,status,grade
0,100,남,유료회원,초1학년
1,1000,여,유료회원,초5학년
2,10000,여,유료회원,초6학년
3,100007,남,유료회원,초4학년
4,10001,남,유료회원,초2학년


In [8]:
JSON_KEY_PATH = "/home/sprint_test/DataEngineeringRepo/03_gcp_python/sa_key/sprintda05_DE_key.json"
pd.read_csv(
    filepath_or_buffer="gs://sprintda05-hojae-bucket/edu_dataset/point_his.csv",
    storage_options={"token":JSON_KEY_PATH}
)

,idx,proc_ym,proc_ymd,point
0,87376,202304,20230401,1000
1,87599,202304,20230401,1000
2,87682,202304,20230401,1000
3,87555,202304,20230401,1000
4,87569,202304,20230401,1000
...,...,...,...,...
368311,97018,202306,20230630,5000
368312,97035,202306,20230630,5000
368313,96869,202306,20230630,5000
368314,96751,202306,20230630,5000


In [9]:
pd.read_parquet(
    path="gs://sprintda05-hojae-bucket/bigquery_data/regdate.parquet",
    storage_options={"token":JSON_KEY_PATH}
)

,idx,regdate
0,1,20221206
1,2,20221206
2,3,20221206
3,4,20221206
4,5,20221206
...,...,...
112563,112565,20231118
112564,112566,20231118
112565,112567,20231118
112566,112568,20231118


## 6. (빅)데이터 저장시 Parquet 파일을 사용하는 이유

- 용량! -> csv에 비해 훨씬 작은 용량을 차지
- 비용! -> 같은 데이터 저장시 비용이 덜 발생
- metadata 같이 저장 -> 저장/읽을 떄 더 빠르고 편리
- 빅데이터 관련 도구들과 호환 -> kafka, spark, flink, trino, hive, iceberg 등 빅데이터 도구들과 높은 호환성
- 파티셔닝 기능 -> 객체 저장소에 저장될 떄 자동으로 파티셔닝이 가능!

In [ ]:
JSON_KEY_PATH = "/home/sprint_test/DataEngineeringRepo/03_gcp_python/sa_key/sprintda05_DE_key.json"

pd.read_csv(
    filepath_or_buffer="gs://sprintda05-hojae-bucket/gcs_part_csv", # 상위 디렉토리 단위로 조회를 하면? -> 에러가 남; 세상에마상에
    storage_options={"token":JSON_KEY_PATH}
)

In [ ]:
pd.read_parquet(
    path="gs://sprintda05-hojae-bucket/gcs_part_parquet", # parquet은 어떨ㄹ까? -> 상위 폴더가 조회가됨!!! 이것도 세상에 마상에
    storage_options={"token":JSON_KEY_PATH}
)

,idx,proc_ym,proc_ymd,pointnm,yyyy,mm,dd
0,75178,202304,2023-04-01,한글 스피치,2023,4,1
1,7287,202304,2023-04-01,중학 1학년,2023,4,1
2,5547,202304,2023-04-01,중학 1학년,2023,4,1
3,80092,202304,2023-04-01,중학 1학년,2023,4,1
4,67940,202304,2023-04-01,중학 2학년,2023,4,1
...,...,...,...,...,...,...,...
44417,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44418,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44419,12257,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
44420,66583,202304,2023-04-30,밀크T 지오그래픽,2023,4,30
